<a href="https://colab.research.google.com/github/ktam069/Google_Colab_Audiovisual/blob/master/AV_Layers_on_CPU_Audio_Only_No_BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive to access npy dataset of audio input for training/testing

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Main program - Part IV Project 80 Basic Version

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, TimeDistributed, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
import os
# import sys
import glob
from datetime import datetime

In [5]:
# Main program - Part IV Project 80 Basic Version

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

import tensorflow as tf
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Conv2D, LSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, Flatten, TimeDistributed, Reshape
from keras.callbacks import ModelCheckpoint
from keras.layers import concatenate
from keras.layers import Lambda

import os
# import sys
import glob
from datetime import datetime

# ===== Settings =====

# Display options
PRINT_DATA = False
DISPLAY_GRAPHS = False

# Data transformation/processing
NORMALISE_DATA = False
POWER_ENCODE = True

# Use float32 format for dataset ndarrays
USE_FLOAT32 = True

# Range of data to use for training (excludes END_ID)
START_ID = 0
END_ID = 21

# Sampling rate
SAMPLING_RATE = 16000

# Filepaths to the locations for input audio-visual files
# path_to_data = "./data/"
path_to_data_audio = "./data/audio/audio_train/"
path_to_data_video = "./data/video/face_input/"
# audio_data_name = "trim_audio_train%d.wav"

# Filepaths to the locations for saved data and models
path_to_models = "./saved_models/"
path_to_saved_datasets = "./dataset_npy/"
path_to_outputs = "./output_wavs/"

# ====================

'''def convolution_model(num_speakers=2):'''

num_speakers=2

# == Audio convolution layers ==

model = Sequential()

# # Implicit input layer
# inputs = Input(shape=(298, 257, 2))
# model.add(inputs)

# Convolution layers
conv1 = Conv2D(96, kernel_size=(1,7), padding='same', dilation_rate=(1,1), input_shape=(298, 257, 2), name="input_layer")
model.add(conv1)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv2 = Conv2D(96, kernel_size=(7,1), padding='same', dilation_rate=(1,1))
model.add(conv2)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv3 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(1,1))
model.add(conv3)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv4 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(2,1))
model.add(conv4)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv5 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(4,1))
model.add(conv5)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv6 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(8,1))
model.add(conv6)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv7 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(16,1))
model.add(conv7)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv8 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(32,1))
model.add(conv8)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv9 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(1,1))
model.add(conv9)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv10 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(2,2))
model.add(conv10)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv11 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(4,4))
model.add(conv11)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv12 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(8,8))
model.add(conv12)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv13 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(16,16))
model.add(conv13)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv14 = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(32,32))
model.add(conv14)
model.add(BatchNormalization())
model.add(Activation("relu"))

conv15 = Conv2D(8, kernel_size=(1,1), padding='same', dilation_rate=(1,1))
model.add(conv15)
model.add(BatchNormalization())
model.add(Activation("relu"))

# == AV fused neural network ==

# AV fusion step(s)
model.add(TimeDistributed(Flatten()))

# BLSTM
new_matrix_length = 400
model.add(Bidirectional(LSTM(new_matrix_length//2, return_sequences=True, input_shape=(298, 257*8))))

# Fully connected layers
model.add(Dense(600, activation="relu"))
model.add(Dense(600, activation="relu"))
model.add(Dense(600, activation="relu"))

# Output layer (i.e. complex masks)
# outputs = Dense(257*2*num_speakers, activation="relu")
outputs = Dense(257*2*num_speakers, activation="sigmoid")				# TODO: check if this is more correct (based on the paper)
model.add(outputs)
outputs_complex_masks = Reshape((298, 257, 2, num_speakers), name="output_layer")
model.add(outputs_complex_masks)

# Print the output shapes of each model layer
for layer in model.layers:
  name = layer.get_config()["name"]
  if "batch_normal" in name or "activation" in name:
    continue
  print(layer.output_shape, "\t", name)

# Alternatively, print the default keras model summary
model.summary()

# Compile the model before training
# model.compile(optimizer='adam', loss='mse')
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


Using TensorFlow backend.



(None, 298, 257, 96) 	 input_layer
(None, 298, 257, 96) 	 conv2d_1
(None, 298, 257, 96) 	 conv2d_2
(None, 298, 257, 96) 	 conv2d_3
(None, 298, 257, 96) 	 conv2d_4
(None, 298, 257, 96) 	 conv2d_5
(None, 298, 257, 96) 	 conv2d_6
(None, 298, 257, 96) 	 conv2d_7
(None, 298, 257, 96) 	 conv2d_8
(None, 298, 257, 96) 	 conv2d_9
(None, 298, 257, 96) 	 conv2d_10
(None, 298, 257, 96) 	 conv2d_11
(None, 298, 257, 96) 	 conv2d_12
(None, 298, 257, 96) 	 conv2d_13
(None, 298, 257, 8) 	 conv2d_14
(None, 298, 2056) 	 time_distributed_1
(None, 298, 400) 	 bidirectional_1
(None, 298, 600) 	 dense_1
(None, 298, 600) 	 dense_2
(None, 298, 600) 	 dense_3
(None, 298, 1028) 	 dense_4
(None, 298, 257, 2, 2) 	 output_layer
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Conv2D)         (None, 298, 257, 96)      1440      
_________________________________________________________________
batch

In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/dataset_train_0-20.npy' "dataset_train_0-20.npy"

In [7]:
!ls

dataset_train_0-20.npy	gdrive	sample_data


In [0]:
dataset_train = np.load("dataset_train_0-20.npy")

In [0]:
# Split dataset into inputs and ground truths
x_train = dataset_train[:,:,:,:,0 ]
y_train = dataset_train[:,:,:,:,1:]


x_visual = np.random.rand(len(x_train), 75, 1, 1024)

In [0]:
import time

In [12]:
times = []
num_tests = 10

for i in range(num_tests):
	'''Predict data'''
	start_time = time.time()
	model.predict(x_train[i:i+1])
	#model.predict([x_train[i:i+1], x_visual[i:i+1], x_visual[i:i+1]])
	end_time = time.time()
	runtime = end_time-start_time
	times.append(runtime)
	print("Time taken for run %d:"%(i+1), runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Time taken for run 1: 9.00955057144165
Time taken for run 2: 8.95258617401123
Time taken for run 3: 8.976745367050171
Time taken for run 4: 8.94874119758606
Time taken for run 5: 8.996212720870972
Time taken for run 6: 8.913541316986084
Time taken for run 7: 8.96448302268982
Time taken for run 8: 8.95845365524292
Time taken for run 9: 8.954936504364014
Time taken for run 10: 9.018389463424683
Number of test: 10
Test times: [9.00955057144165, 8.95258617401123, 8.976745367050171, 8.94874119758606, 8.996212720870972, 8.913541316986084, 8.96448302268982, 8.95845365524292, 8.954936504364014, 9.018389463424683]
Average running time: 8.96936399936676


In [0]:
fname = "ablation_basic_times_ao_noBLSTM.csv"
np.savetxt(fname, times, delimiter=",")

In [0]:
new_dir = "/content/gdrive/My Drive/Part IV Project/Colab Notebooks/AV_basic_ablation_output/"
if not os.path.isdir(new_dir):
  os.mkdir(new_dir)

In [0]:
!cp $fname "/content/gdrive/My Drive/Part IV Project/Colab Notebooks/AV_basic_ablation_output/${fname}.csv"

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Conv2D)         (None, 298, 257, 96)      1440      
_________________________________________________________________
batch_normalization_1 (Batch (None, 298, 257, 96)      384       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 257, 96)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 298, 257, 96)      64608     
_________________________________________________________________
batch_normalization_2 (Batch (None, 298, 257, 96)      384       
_________________________________________________________________
activation_2 (Activation)    (None, 298, 257, 96)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 298, 257, 96)     